In [1]:
import os
from sklearn import metrics
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import io
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
#Need to hardcode the location so Jupyter can find the files
train_set = 'C:/Users/David/Desktop/Tensorflow/UNSW Data/UNSW_NB15_training-set.csv'
test_set = 'C:/Users\David/Desktop/Tensorflow/UNSW Data/UNSW_NB15_testing-set.csv'

training = pd.read_csv(train_set, index_col='id')
test = pd.read_csv(test_set, index_col='id')

data = pd.concat([training, test]) # Just merging the sets for now as training doesn't match test columns when ONEHOT
                                   # And this is just easier for now when testing stuff
data_cols = data.columns.drop('label')
data = data[data_cols]

In [3]:
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)

In [4]:
##Looking for attack_cat
CATEGORICAL_COLUMNS = ['proto', 'service', 'state']
NUMERIC_COLUMNS = ['dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl', 'dttl', 'sload',
                   'dload', 'sloss', 'dloss', 'sinpkt', 'dinpkt', 'sjit', 'djit', 'swin', 'stcpb',
                   'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'trans_depth',
                   'response_body_len', 'ct_state_ttl', 'ct_flw_http_mthd', 'is_ftp_login',
                   'ct_ftp_cmd', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ltm',
                   'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'is_sm_ips_ports']

for col in CATEGORICAL_COLUMNS:
    encode_text_dummy(data, col)

for col in NUMERIC_COLUMNS:
    encode_numeric_zscore(data, col)

training.dropna(inplace=True, axis=1)
test.dropna(inplace=True, axis=1)
print(training[0:5])

         dur proto service state  spkts  dpkts  sbytes  dbytes       rate  \
id                                                                          
1   0.121478   tcp       -   FIN      6      4     258     172  74.087490   
2   0.649902   tcp       -   FIN     14     38     734   42014  78.473372   
3   1.623129   tcp       -   FIN      8     16     364   13186  14.170161   
4   1.681642   tcp     ftp   FIN     12     12     628     770  13.677108   
5   0.449454   tcp       -   FIN     10      6     534     268  33.373826   

    sttl  ...  ct_dst_sport_ltm  ct_dst_src_ltm  is_ftp_login  ct_ftp_cmd  \
id        ...                                                               
1    252  ...                 1               1             0           0   
2     62  ...                 1               2             0           0   
3     62  ...                 1               3             0           0   
4     62  ...                 1               3             1           1  

In [5]:
x_columns = data.columns.drop('attack_cat')
x = data[x_columns].values
dummies = pd.get_dummies(data['attack_cat'])  # Classification
outcomes = dummies.columns
num_classes = len(outcomes)
y = dummies.values

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.15, random_state=42)


In [6]:
model = Sequential()
model.add(Dense(15, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(30, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(10, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
# model.add(Dense(1, kernel_initializer='normal'))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
print(model.summary())
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
model.fit(x_train, y_train, validation_data=(x_test,y_test),callbacks=[monitor], verbose=2, epochs=50)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 15)                2955      
_________________________________________________________________
dense_1 (Dense)              (None, 30)                480       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                310       
_________________________________________________________________
dense_3 (Dense)              (None, 10)                110       
Total params: 3,855
Trainable params: 3,855
Non-trainable params: 0
_________________________________________________________________
None
Train on 219022 samples, validate on 38651 samples
Epoch 1/50
219022/219022 - 7s - loss: 0.6797 - val_loss: 0.5783
Epoch 2/50
219022/219022 - 6s - loss: 0.5591 - val_loss: 0.5509
Epoch 3/50
219022/219022 - 6s - loss: 0.5374 - val_loss: 0.5323
Epoc

In [7]:
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)
y_eval = np.argmax(y_test,axis=1)
score = metrics.accuracy_score(y_eval, pred)
print("Validation score: {}".format(score))


Validation score: 0.8057230084603244
